In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
import numpy as np
import pandas as pd 

data_path = '/content/drive/MyDrive/태양광 발전량 예측 AI 경진대회/235680_태양광 발전량 예측 AI 경진대회_data/'

train = pd.read_csv(data_path + 'train/train.csv')

In [91]:
#hour angle 추가
h_list=[]
len(train['Hour'])
for i in range(len(train['Hour'])):
  h=train.loc[i,'Hour']
  min=train.loc[i,'Minute']
  if min!=30:
    hour_angle=15*(h-12)-7.5
    h_list.append(hour_angle)
  else:
    hour_angle=15*(h-12)
    h_list.append(hour_angle)

train['h']=h_list

In [92]:
# Time 변수 생성 
df_train=pd.read_csv(data_path+'train/train.csv')
df_train['Time'] = df_train['Hour'] * 60 + df_train['Minute']
df_train.to_csv('df_train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52560 entries, 0 to 52559
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Day     52560 non-null  int64  
 1   Hour    52560 non-null  int64  
 2   Minute  52560 non-null  int64  
 3   DHI     52560 non-null  int64  
 4   DNI     52560 non-null  int64  
 5   WS      52560 non-null  float64
 6   RH      52560 non-null  float64
 7   T       52560 non-null  int64  
 8   TARGET  52560 non-null  float64
 9   Time    52560 non-null  int64  
dtypes: float64(3), int64(7)
memory usage: 4.0 MB


In [93]:
# 날짜별 데이터프레임 쪼개기
df_train_day = []
list_day = [] 
DAY = 0

for n in range(0,df_train.shape[0]):

    if df_train.loc[n, 'Day'] == DAY:
        list_day.append(df_train.loc[n, ['Time','TARGET']])
        
    else:
        df_day = pd.DataFrame(list_day, columns = ['Time', 'TARGET'], index = range(0,48))
        df_train_day.append(df_day)

        list_day = []
        list_day.append(df_train.loc[n, ['Time','TARGET']])
    DAY = df_train.loc[n, 'Day']



In [94]:
# 일출시간, 일몰시간 구하기
df_new = pd.DataFrame(columns = ['Day', '일출시간', '일몰시간', '일조시간'])

for i in range(0,len(df_train_day)):
    for d1 in range(0,48):
        if df_train_day[i].loc[d1,'TARGET'] != 0:
            sunrise = df_train_day[i].loc[d1,'Time']
            break

    for d2 in range(47,-1,-1):
        if df_train_day[i].loc[d2,'TARGET'] != 0:
            sunset = df_train_day[i].loc[d2 + 1,'Time']
            break
        
    df_new.loc[i] = [i, sunrise, sunset, sunset - sunrise]

df_new.to_csv('df_new.csv')

,Day,일출시간,일몰시간,일조시간
0,0.0,480.0,1020.0,540.0
1,1.0,480.0,1020.0,540.0
2,2.0,480.0,1020.0,540.0
3,3.0,480.0,1020.0,540.0
4,4.0,480.0,1020.0,540.0


In [95]:
# 하지, 동지 구하기

summer = df_new.loc[df_new['일조시간'].idxmax(), 'Day']
winter = df_new.loc[df_new['일조시간'].idxmin(), 'Day']

print(summer, winter)

# 138 + 195 = 233 -> 1월 1일

138.0 356.0


In [161]:
N=[]
#1월 1일 기준 N+171 = 하지 기준 N
for i in range(len(train['Day'])):
  day=train.loc[i,'Day']
  if day==summer:
    N.append(+171)
  else:
    N.append(day-summer+171)


#N Feature 생성
#N이 음수, 364 초과를 변환(1월 1일 기준 N=0)
train['N']=N
day_max=train['N'].max()
day_min=train['N'].min()

while day_max>364 or day_max<0:
  for i in range(len(train['N'])):
    day=train.loc[i,'N']
    if day<0:
      train.loc[i,'N']=day+365
    elif day>364:
      train.loc[i,'N']=day-365
    else:
      pass
  day_max=train['N'].max()
  day_min=train['N'].min()

#현재 N은 한국 기준이므로, UT로 변환
train['N']-=0.375


In [162]:
#Declination of the Sun

def radian_angle(x):
  return x*np.pi/180

def dec_sun(N):
  return -1*np.arcsin(0.39779*np.cos(radian_angle(0.98565)*(N+10)+radian_angle(1.914)*np.sin(radian_angle(0.98565)*(N-2))))

train['Declination']=dec_sun(train['N'])

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,N,Declination,Zenith_Angle,GHI
0,0,0,0,0,0,1.5,69.08,-12,0.0,-187.5,32.625,-0.295314,-0.873733,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0,-180.0,32.625,-0.295314,-0.880429,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0,-172.5,32.625,-0.295314,-0.873733,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0,-165.0,32.625,-0.295314,-0.853761,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0,-157.5,32.625,-0.295314,-0.820853,0.0


In [163]:
#Zenith Angle

#관측자 위도 설정
alt=37.54

def zenith_angle(altitude,declination,hour_angle):
  return np.sin(radian_angle(altitude))*np.sin(declination)+np.cos(radian_angle(altitude))*np.cos(declination)*np.cos(radian_angle(hour_angle))

train['Zenith_Angle']=zenith_angle(alt,train['Declination'],train['h'])

In [171]:
#GHI

train['GHI']=train['DHI']+train['DNI']*train['Zenith_Angle']

        Day  Hour  Minute    DHI    DNI   WS      RH     T     TARGET      h  \
6624  138.0   0.0     0.0    0.0    0.0  3.5  100.00   6.0   0.000000 -187.5   
6625  138.0   0.0    30.0    0.0    0.0  3.3  100.00   5.0   0.000000 -180.0   
6626  138.0   1.0     0.0    0.0    0.0  3.1   99.26   5.0   0.000000 -172.5   
6627  138.0   1.0    30.0    0.0    0.0  3.0  100.00   5.0   0.000000 -165.0   
6628  138.0   2.0     0.0    0.0    0.0  3.0  100.00   5.0   0.000000 -157.5   
6629  138.0   2.0    30.0    0.0    0.0  2.9  100.00   5.0   0.000000 -150.0   
6630  138.0   3.0     0.0    0.0    0.0  2.9  100.00   5.0   0.000000 -142.5   
6631  138.0   3.0    30.0    0.0    0.0  2.4  100.00   4.0   0.000000 -135.0   
6632  138.0   4.0     0.0    0.0    0.0  1.9  100.00   4.0   0.000000 -127.5   
6633  138.0   4.0    30.0    0.0    0.0  1.9  100.00   3.0   0.000000 -120.0   
6634  138.0   5.0     0.0    4.0    0.0  1.9  100.00   3.0   0.375379 -112.5   
6635  138.0   5.0    30.0   20.0    0.0 